<a href="https://colab.research.google.com/github/jwang44/crispy-fiesta/blob/main/feature_binary_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


## Load the data and get basic features

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('./train.csv',engine='python')
test = pd.read_csv('./test.csv',engine='python')

In [4]:
X_train = train.body  # train texts
y_train = train.subreddit # train subreddits
X_test = test.body  # test texts

In [5]:
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [6]:
# transform target labels to values
le = LabelEncoder()
y_train_num = le.fit_transform(y_train.values) # convert category from string to numerical (!!!!! update the variables in kcross fold)

# vectorize word count
vectorizer = CountVectorizer()
vectors_train = vectorizer.fit_transform(X_train)
vectors_test = vectorizer.transform(X_test)

# print(vectorizer.get_feature_names())
print(vectors_train.shape)
print(vectors_test.shape)

(1999, 15365)
(1378, 15365)


In [7]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### features:vectors_train_stop_Lemma, vectors_test_stop_Lemma

In [10]:
# put it all together: remove stop words and punctuation, lemmatization
stop_words = text.ENGLISH_STOP_WORDS

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]

vectorizer = CountVectorizer(binary = True, stop_words = stop_words, tokenizer = New_LemmaTokenizer())
vectors_train_stop_Lemma = vectorizer.fit_transform(X_train)
vectors_test_stop_Lemma = vectorizer.transform(X_test)

# print(vectorizer.get_feature_names())
print(vectors_train_stop_Lemma.shape)
print(vectors_test_stop_Lemma.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(1999, 9779)
(1378, 9779)


### features:vectors_train_stop_stem, vectors_test_stop_stem

In [ ]:
# remove stopwords and punctuation, stemming, 
stop_words = text.ENGLISH_STOP_WORDS

class StemTokenizer:
     def __init__(self):
       self.wnl =PorterStemmer()
     def __call__(self, doc):
       return [self.wnl.stem(t) for t in word_tokenize(doc) if t.isalpha()]

vectorizer = CountVectorizer(binary = True, stop_words = stop_words, tokenizer=StemTokenizer())
vectors_train_stop_stem = vectorizer.fit_transform(X_train)
vectors_test_stop_stem = vectorizer.transform(X_test)
print(vectors_train_stop_stem.shape)
print(vectors_test_stop_stem.shape)

## Feature selection

In [15]:
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold, f_classif, mutual_info_classif, RFE, SelectFromModel

### 13.2.1 chi2  features:vectors_train_Lemma_X2, vectors_test_Lemma_X2,   
###              vectors_train_stem_X2, vectors_test_stem_X2

In [21]:
select = SelectKBest(chi2, k=5000)
vectors_train_Lemma_X2 = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_Lemma_X2 = select.transform(vectors_test_stop_Lemma)
print(vectors_train_Lemma_X2.shape)

(1999, 5000)


In [22]:
select = SelectKBest(chi2, k=4000)
vectors_train_stem_X2 = select.fit_transform(vectors_train_stop_stem, y_train_num)
vectors_test_stem_X2 = select.transform(vectors_test_stop_stem)
print(vectors_train_stem_X2.shape)

(1999, 4000)


### 13.2.2 mutual info   features:vectors_train_Lemma_mutual vectors_test_Lemma_mutual,   
###vectors_train_stem_mutual, vectors_test_stem_mutual

In [19]:
select = SelectKBest(mutual_info_classif, k=5000)
vectors_train_Lemma_mutual = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_Lemma_mutual = select.transform(vectors_test_stop_Lemma)
print(vectors_train_Lemma_mutual.shape)

(1999, 5000)


In [20]:
select = SelectKBest(mutual_info_classif, k=4000)
vectors_train_stem_mutual = select.fit_transform(vectors_train_stop_stem, y_train_num)
vectors_test_stem_mutual = select.transform(vectors_test_stop_stem)
print(vectors_train_stem_mutual.shape)

(1999, 4000)


### 13.2.3 F score  features:vectors_train_Lemma_F, vectors_test_Lemma_F,   
###vectors_train_stem_F, vectors_test_stem_F

In [17]:
select = SelectKBest(f_classif, k=6000)
vectors_train_Lemma_F = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_Lemma_F = select.transform(vectors_test_stop_Lemma)
print(vectors_train_Lemma_F.shape)

(1999, 6000)


In [18]:
select = SelectKBest(f_classif, k=5000)
vectors_train_stem_F = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_stem_F = select.transform(vectors_test_stop_Lemma)
print(vectors_train_stem_F.shape)

(1999, 5000)


### 13.3 Recursive feature elimination features:vectors_train_Lemma_RFE, vectors_test_Lemma_RFE,   
###vectors_train_stem_RFE, vectors_test_stem_RFE

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.model_selection import KFold, cross_val_score

In [16]:
estimator = LogisticRegression()
select = RFE(estimator, n_features_to_select=5000, step=0.1)
vectors_train_Lemma_RFE = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_Lemma_RFE = select.transform(vectors_test_stop_Lemma)
print(vectors_train_Lemma_RFE.shape)

(1999, 5000)


In [ ]:
estimator = LogisticRegression()
select = RFE(estimator, n_features_to_select=5000, step=0.1)
vectors_train_stem_RFE = select.fit_transform(vectors_train_stop_stem, y_train_num)
vectors_test_stem_RFE = select.transform(vectors_test_stop_stem)

### 13.4 selectfrommodel

In [ ]:
estimator = LinearSVC(C=0.01, penalty="l1",dual=False)
select = SelectFromModel(estimator,max_features=5000)
vectors_train_Lemma_SFM = select.fit_transform(vectors_train_stop_Lemma, y_train_num)
vectors_test_Lemma_SFM = select.transform(vectors_test_stop_Lemma)
print(vectors_train_Lemma_SFM.shape)

(1999, 1)


In [ ]:
estimator = LinearSVC(C=0.01, penalty="l1",dual=False)
select = SelectFromModel(estimator,max_features=5000)
vectors_train_stem_SFM = select.fit_transform(vectors_train_stop_stem, y_train_num)
vectors_test_stem_SFM = select.transform(vectors_test_stop_stem)
print(vectors_train_stem_SFM.shape)

(1999, 1)


#### Write results to CSV

In [ ]:
result = pd.DataFrame({'id': test.id, 'subreddit': y_pred})
result.to_csv("result.csv", index=False)

In [ ]:
pred_csv = pd.read_csv('result.csv',engine='python')
pred_csv.head()

,id,subreddit
0,0,science
1,1,science
2,2,science
3,3,science
4,4,science
